In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv("train.csv")
train.head(3)

In [ ]:
test = pd.read_csv("test.csv")
test.head(3)

In [ ]:
pd.pivot_table(
    train,
    index="Pclass",
    columns="Survived",
    aggfunc="mean",
)

In [ ]:
pd.pivot_table(
    train,
    index="Sex",
    columns="Survived",
    aggfunc="mean",
)

In [ ]:
pd.pivot_table(
    train,
    index="SibSp",
    columns="Survived",
    aggfunc="mean",
    fill_value=0,
)

In [ ]:
pd.pivot_table(
    train,
    index="Parch",
    columns="Survived",
    aggfunc="mean",
    fill_value=0,
)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
survived_0 = train.loc[train["Survived"] == 0]
survived_1 = train.loc[train["Survived"] == 1]


sns.distplot(survived_0["Age"], label="survived_0");
sns.distplot(survived_1["Age"], label="survived_1");

plt.legend();

In [ ]:
pclass_1_survived_0 = train.loc[(train["Pclass"] == 1) & (train["Survived"] == 0)]
pclass_1_survived_1 = train.loc[(train["Pclass"] == 1) & (train["Survived"] == 1)]

pclass_2_survived_0 = train.loc[(train["Pclass"] == 2) & (train["Survived"] == 0)]
pclass_2_survived_1 = train.loc[(train["Pclass"] == 2) & (train["Survived"] == 1)]

pclass_3_survived_0 = train.loc[(train["Pclass"] == 3) & (train["Survived"] == 0)]
pclass_3_survived_1 = train.loc[(train["Pclass"] == 3) & (train["Survived"] == 1)]


In [ ]:
sns.distplot(pclass_1_survived_0["Age"], label="pclass_1_survived_0");
sns.distplot(pclass_1_survived_1["Age"], label="pclass_1_survived_1");
plt.legend();

In [ ]:
sns.distplot(pclass_2_survived_0["Age"], label="pclass_2_survived_0");
sns.distplot(pclass_2_survived_1["Age"], label="pclass_2_survived_1");
plt.legend();

In [ ]:
sns.distplot(pclass_3_survived_0["Age"], label="pclass_3_survived_0");
sns.distplot(pclass_3_survived_1["Age"], label="pclass_3_survived_1");
plt.legend();

In [ ]:
sns.pointplot(data=train, x="Pclass", y="Survived", hue="Sex");

In [ ]:
embarked_S = train.loc[train["Embarked"] == "S"]
embarked_C = train.loc[train["Embarked"] == "C"]
embarked_Q = train.loc[train["Embarked"] == "Q"]

In [ ]:
sns.pointplot(data=embarked_S, x="Pclass", y="Survived", hue="Sex");
plt.title("Embarked = S");

In [ ]:
sns.pointplot(data=embarked_C, x="Pclass", y="Survived", hue="Sex");
plt.title("Embarked = C");

In [ ]:
sns.pointplot(data=embarked_Q, x="Pclass", y="Survived", hue="Sex");
plt.title("Embarked = Q");

In [ ]:
sns.barplot(data=embarked_S, x="Sex", y="Fare", hue="Survived");
plt.title("Embarked = S");

In [ ]:
sns.barplot(data=embarked_C, x="Sex", y="Fare", hue="Survived");
plt.title("Embarked = C");

In [ ]:
sns.barplot(data=embarked_Q, x="Sex", y="Fare", hue="Survived");
plt.title("Embarked = Q");

In [ ]:
train.head(3)

In [ ]:
test.head(3)

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train["Age"].fillna(train["Age"].median(), inplace=True)
train["Embarked"].fillna(train["Embarked"].mode()[0], inplace=True)

test["Age"].fillna(test["Age"].median(), inplace=True)
test["Fare"].fillna(test["Fare"].median(), inplace=True)

drop_columns = ["PassengerId", "Cabin", "Ticket"]
train = train.drop(drop_columns, axis=1)
test = test.drop(drop_columns, axis=1)

print(train.isnull().sum())
print("-" * 10)
print(test.isnull().sum())

In [ ]:
train["family_size"] = train["SibSp"] + train["Parch"] + 1
test["family_size"] = test["SibSp"] + test["Parch"] + 1

train["is_single"] = 1
train["is_single"].loc[train["family_size"] > 1] = 0
test["is_single"] = 1
test["is_single"].loc[test["family_size"] > 1] = 0

train["title"] = train["Name"].str.split(",", expand=True)[1].str.split(".", expand=True)[0]
test["title"] = test["Name"].str.split(",", expand=True)[1].str.split(".", expand=True)[0]

train["fare_bin"] = pd.qcut(x=train["Fare"], q=4)
test["fare_bin"] = pd.qcut(x=test["Fare"], q=4)

train["age_bin"] = pd.cut(x=train["Age"], bins=5)
test["age_bin"] = pd.cut(x=test["Age"], bins=5)

print(train.shape, test.shape)
train.head(3)

In [ ]:
min_val = 10

train_title = train["title"].value_counts() < min_val
test_title = test["title"].value_counts() < min_val

train["title"] = train["title"].apply(lambda x: "Other" if train_title.loc[x] == True else x)
test["title"] = test["title"].apply(lambda x: "Other" if test_title.loc[x] == True else x)

print(train["title"].value_counts())
print("-" * 10)
print(test["title"].value_counts())

In [ ]:
train.head(3)

In [ ]:
from sklearn import preprocessing

features = ["Sex", "Embarked", "title", "fare_bin", "age_bin"]

for feat in features:
    lbl_enc = preprocessing.LabelEncoder()
    train.loc[:, feat] = lbl_enc.fit_transform(train[feat])
    test.loc[:, feat] = lbl_enc.fit_transform(test[feat])
    

In [ ]:
train.head()

In [ ]:
train = train.drop("Name", axis=1)
test = test.drop("Name", axis=1)

In [ ]:
sns.countplot(train["Survived"]);

In [ ]:
from sklearn import model_selection

train["kfold"] = -1

df = train.sample(frac=1).reset_index(drop=True)
y = train.Survived.values

skf = model_selection.StratifiedKFold(n_splits=5)

for fold, (trn_idx, val_idx) in enumerate(skf.split(X=df, y=y)):
    df.loc[val_idx, "kfold"] = fold

df.to_csv("train_folds.csv", index=False)

In [ ]:
import joblib
from sklearn import metrics
from sklearn import linear_model

def run(fold):
    df = pd.read_csv("train_folds.csv")
    
    train_df = df[df.kfold != fold].reset_index(drop=True)
    valid_df = df[df.kfold == fold].reset_index(drop=True)
    
    xtrain = train_df.drop("Survived", axis=1).values
    ytrain = train_df.Survived.values
    
    xvalid = valid_df.drop("Survived", axis=1).values
    yvalid = valid_df.Survived.values
    
    clf = linear_model.LogisticRegression()
    clf.fit(xtrain, ytrain)
    
    preds = clf.predict(xvalid)
    acc = metrics.accuracy_score(preds, yvalid)
    
    print(f"Fold {fold}, Accuracy={acc:.3f}")
    
    joblib.dump(
        clf, 
        f"model_{fold}.bin",
    )
    

In [ ]:
for fold_ in range(5):
    run(fold_)

In [ ]:
df = pd.read_csv("train_folds.csv")
clf_columns = list(df.columns)
clf_columns